In [6]:
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity

# 1. 数据读取与基本预处理
df = pd.read_excel("result.xlsx")

# 清洗列名：去除前后空格
df.columns = df.columns.str.strip()

# 查看列名，便于检查是否匹配
print("所有列名：")
print(df.columns.tolist())

# 2. 数据预处理
# 此处假设问卷中 Likert 量表部分已经是数值型，例如题目10和题目11系列
# 如果部分题目仍为字符串，则需要建立映射字典进行转换（此处未做转换）

# 检查“所用时间”和“总分”字段的描述统计，确认数据单位与范围
print("【所用时间】描述统计：")
print(df["所用时间"].describe())
print("【总分】描述统计：")
print(df["总分"].describe())

# 3. 信度分析（内部一致性）
# 分为两部分：① 痛点问题部分（题目10系列） ② 平台功能认可度部分（题目11系列）

# 根据最新问卷格式（请确保下面列名与您的实际文件一致），例如：
pain_columns = ['10、请根据您的体验，对于下列问题选择同意程度（1：非常不同意，5：非常同意）：—我认为目前小组作业存在明显的分工不明确、任务重心不均问题。', '10、小组中经常出现搭便车现象，导致团队整体效率降低。', '10、小组作业过程中沟通不畅和缺乏有效的反馈机制，是主要影响效率的因素。', '10、当前教师在评分时仅依据作业最终结果，忽略过程中的努力与协作，导致评分偏差。', '10、组长在组织和管理小组作业中面临过多协调和沟通压力，影响工作质量。', '10、缺乏实时监控与进度跟踪使得小组作业的整体过程缺乏透明性和公正性。']

platform_columns = ['11、请您根据对平台功能的理解，选择同意程度（1：非常不同意，5：非常同意）：—我认为引入AI进行智能任务分工，可以有效降低分工不均和搭便车现象。', '11、使用AI辅助对小组作业过程和成果进行质量分析，将使教师与组长更全面地了解团队情况。', '11、基于AI推荐的组队和组长选择，可以大大缓解当前组长压力和提高小组作业效率。', '11、智能合作平台能够提高小组作业的透明度和公正性，从而提升学生对小组作业的满意度。', '11、总体上，我对这种智能合作平台持认可态度，并认为它会提升我参与小组作业的积极性和热情。']

# 预处理：检查关键列是否有缺失值（这里不做删除，仅查看）
print("痛点问题部分缺失率：")
print(df[pain_columns].isnull().mean())
print("平台功能认可度部分缺失率：")
print(df[platform_columns].isnull().mean())

# 4. 定义 Cronbach’s α 计算函数
def cronbach_alpha(items_scores):
    items = items_scores.dropna()  # 只使用非缺失值的记录
    item_variances = items.var(axis=0, ddof=1)
    total_var = items.sum(axis=1).var(ddof=1)
    n_items = items.shape[1]
    if total_var == 0:
        return np.nan
    alpha = (n_items / (n_items - 1)) * (1 - item_variances.sum() / total_var)
    return alpha

# 计算痛点问题部分的 Cronbach’s α
alpha_pain = cronbach_alpha(df[pain_columns])
print("\n【信度分析】痛点问题部分的 Cronbach’s alpha 值：", alpha_pain)

# 计算平台功能认可度部分的 Cronbach’s α
alpha_platform = cronbach_alpha(df[platform_columns])
print("【信度分析】平台功能认可度部分的 Cronbach’s alpha 值：", alpha_platform)

# 5. 效度初步检验：探索性因素分析（EFA）针对平台功能认可度部分
# 检查数据适合性（KMO与Bartlett检验）
data_platform = df[platform_columns].dropna()
kmo_all, kmo_model = calculate_kmo(data_platform)
bartlett_chi, bartlett_p = calculate_bartlett_sphericity(data_platform)
print("\n【效度分析】平台功能认可度部分：")
print("KMO值：", kmo_model)
print("Bartlett检验 p-value：", bartlett_p)

# 如果KMO值>=0.6且Bartlett检验显著(p<0.05)，则适合因素分析
# 这里设定提取1个因子（也可尝试提取多个，根据碎石图或解释方差决定）
fa = FactorAnalyzer(n_factors=1, rotation="varimax")
fa.fit(data_platform)
loadings = fa.loadings_
print("因子载荷矩阵：")
print(loadings)

# 6. 输出描述性统计（频率、均值和标准差）以了解问卷回答分布
# 例如，针对“每学期参与小组作业的次数”（假设列名为“4、您每学期参与小组作业的次数大致为:”）
freq_col = "4、您每学期参与小组作业的次数大致为:"
print("\n【描述性统计】“每学期参与小组作业的次数”频率分布：")
print(df[freq_col].value_counts())

# 针对“总分”，计算均值和标准差
print("\n【描述性统计】总分的均值与标准差：")
print("均值：", df["总分"].mean())
print("标准差：", df["总分"].std())

# 针对痛点问题部分的各题目，输出均值、标准差和频率分布
print("\n【描述性统计】痛点问题部分各题目的统计信息：")
for col in pain_columns:
    print(f"题目：{col}")
    mean_val = df[col].mean()
    std_val = df[col].std()
    print("均值：", mean_val, "标准差：", std_val)
    print("频率分布：")
    print(df[col].value_counts().sort_index())
    print("-" * 50)

# 针对平台功能认可度部分题目，输出描述性统计
print("\n【描述性统计】平台功能认可度部分各题目的统计信息：")
for col in platform_columns:
    print(f"题目：{col}")
    mean_val = df[col].mean()
    std_val = df[col].std()
    print("均值：", mean_val, "标准差：", std_val)
    print("频率分布：")
    print(df[col].value_counts().sort_index())
    print("-" * 50)

所有列名：
['序号', '提交答卷时间', '所用时间', '来源', '来源详情', '来自IP', '总分', '1、您的性别:', '2、您目前所在的年级:', '3、您所就读的专业类别:', '4、您每学期参与小组作业的次数大致为:', '5、请您评价当前小组作业的总体安排（任务分工、角色分配）:', '6、您在小组作业过程中是否经常遇到分工不明确、任务不均衡等问题?', '7、关于小组作业过程中出现的搭便车现象，您的体验是:', '8、您认为当前小组作业中组长的工作压力如何?', '9、对于教师在小组作业评分中仅关注最终结果，而忽略过程监控的情况，您认为这种评价方式是否合理?', '10、请根据您的体验，对于下列问题选择同意程度（1：非常不同意，5：非常同意）：—我认为目前小组作业存在明显的分工不明确、任务重心不均问题。', '10、小组中经常出现搭便车现象，导致团队整体效率降低。', '10、小组作业过程中沟通不畅和缺乏有效的反馈机制，是主要影响效率的因素。', '10、当前教师在评分时仅依据作业最终结果，忽略过程中的努力与协作，导致评分偏差。', '10、组长在组织和管理小组作业中面临过多协调和沟通压力，影响工作质量。', '10、缺乏实时监控与进度跟踪使得小组作业的整体过程缺乏透明性和公正性。', '11、请您根据对平台功能的理解，选择同意程度（1：非常不同意，5：非常同意）：—我认为引入AI进行智能任务分工，可以有效降低分工不均和搭便车现象。', '11、使用AI辅助对小组作业过程和成果进行质量分析，将使教师与组长更全面地了解团队情况。', '11、基于AI推荐的组队和组长选择，可以大大缓解当前组长压力和提高小组作业效率。', '11、智能合作平台能够提高小组作业的透明度和公正性，从而提升学生对小组作业的满意度。', '11、总体上，我对这种智能合作平台持认可态度，并认为它会提升我参与小组作业的积极性和热情。', '12、【附加问题】您认为目前小组作业的最大痛点是什么?（可多选）', '13、对于智能合作平台的功能，您还有哪些建议或意见?']
【所用时间】描述统计：
count     157
unique    126
top       31秒
freq        3
Name: 所用时间, dtype: object
【总分】描述统计：

/Users/jm/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/jm/.pyenv/versions/3.10.12/lib/python3.10/site-packages/factor_analyzer/factor_analyzer.py:663: UserWarning: No rotation will be performed when the number of factors equals 1.
  warnings.warn(


In [7]:
# 4. 分析频率分布与描述性统计

# 4.1 分析“每学期参与小组作业的次数”
freq_col = "4、您每学期参与小组作业的次数大致为:"  # 根据实际列名调整
print("\n【每学期参与小组作业的次数】的频率分布：")
print(df[freq_col].value_counts(dropna=False))

# 如果该列为类别型变量，也可以查看其缺失情况：
print("\n缺失比例：")
print(df[freq_col].isnull().mean())

# 4.2 分析“总分”
print("\n【总分】的描述性统计：")
print("均值：", df["总分"].mean())
print("标准差：", df["总分"].std())
print("最小值：", df["总分"].min())
print("最大值：", df["总分"].max())
print("\n【总分】的频率分布：")
print(df["总分"].value_counts().sort_index())

print("\n【痛点问题部分】的描述性统计：")
for col in pain_columns:
    print(f"\n题目：{col}")
    mean_val = df[col].mean()
    std_val = df[col].std()
    print("均值：", mean_val)
    print("标准差：", std_val)
    print("最小值：", df[col].min(), "最大值：", df[col].max())
    print("频率分布：")
    print(df[col].value_counts().sort_index())


print("\n【平台功能认可度部分】的描述性统计：")
for col in platform_columns:
    print(f"\n题目：{col}")
    mean_val = df[col].mean()
    std_val = df[col].std()
    print("均值：", mean_val)
    print("标准差：", std_val)
    print("最小值：", df[col].min(), "最大值：", df[col].max())
    print("频率分布：")
    print(df[col].value_counts().sort_index())


【每学期参与小组作业的次数】的频率分布：
4、您每学期参与小组作业的次数大致为:
B. 2-3次     104
C. 4-6次      35
A. 0-1次      12
D. 7次及以上      6
Name: count, dtype: int64

缺失比例：
0.0

【总分】的描述性统计：
均值： 40.93630573248408
标准差： 5.896197741202746
最小值： 18
最大值： 52

【总分】的频率分布：
总分
18     1
19     1
28     1
30     4
31     5
32     2
33     3
34     3
35     5
36     5
37     8
38    11
39    10
40    12
41    10
42     7
43    11
44    12
45    13
46     4
47     5
48     9
49    10
50     4
52     1
Name: count, dtype: int64

【痛点问题部分】的描述性统计：

题目：10、请根据您的体验，对于下列问题选择同意程度（1：非常不同意，5：非常同意）：—我认为目前小组作业存在明显的分工不明确、任务重心不均问题。
均值： 3.5859872611464967
标准差： 1.1096579860436853
最小值： 1 最大值： 5
频率分布：
10、请根据您的体验，对于下列问题选择同意程度（1：非常不同意，5：非常同意）：—我认为目前小组作业存在明显的分工不明确、任务重心不均问题。
1     6
2    25
3    31
4    61
5    34
Name: count, dtype: int64

题目：10、小组中经常出现搭便车现象，导致团队整体效率降低。
均值： 3.573248407643312
标准差： 1.063399372935506
最小值： 1 最大值： 5
频率分布：
10、小组中经常出现搭便车现象，导致团队整体效率降低。
1     9
2    10
3    52
4    54
5    32
Name: count, dtype: int64

题目：10、小组作业过程中沟通不畅和缺乏有效的反馈机制，是